In [1]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
def closure(synset,fn):
    seen = set()
    node = set()
    edge = list()
    def recurse(s):
        if not s in seen:
            seen.add(s)
            node.add(s.lemma_names('eng')[0])
            for i in range(1, len(s.lemma_names('eng'))):
                node.add(s.lemma_names('eng')[i])
                edge.append({"source":s.lemma_names('eng')[0],"target":s.lemma_names('eng')[i],"type":"synonym"})
            for s1 in fn(s):
                node.add(s1.lemma_names('eng')[0])
                edge.append({"source":s.lemma_names('eng')[0],"target":s1.lemma_names('eng')[0],"type":"hypernym"})
                recurse(s1)
    recurse(synset)
    return (list(node), edge)


In [5]:
import json

cat = wn.synset('cat.n.01')
graph = closure(cat,lambda s: s.hypernyms())
print(graph)
nodes = []
nodesJson = {}
for i in range(len(graph[0])):
    nodes.append({"id":i+1, "name": graph[0][i]})
    nodesJson[graph[0][i]] = i+1
links = []
for i in range(len(graph[1])):
    graph[1][i]['source'] = nodesJson[graph[1][i]['source']]
    graph[1][i]['target'] = nodesJson[graph[1][i]['target']]
    links.append(graph[1][i])
with open("wordnet.json","w") as fh:
    json.dump({"nodes":nodes,"links":graph[1]},fh)

(['felid', 'entity', 'feline', 'object', 'organism', 'animal', 'eutherian', 'eutherian_mammal', 'placental_mammal', 'creature', 'animate_thing', 'mammal', 'chordate', 'true_cat', 'beast', 'cat', 'fauna', 'unit', 'placental', 'carnivore', 'brute', 'being', 'physical_object', 'mammalian', 'whole', 'living_thing', 'physical_entity', 'vertebrate', 'craniate', 'animate_being'], [{'source': 'cat', 'target': 'true_cat', 'type': 'synonym'}, {'source': 'cat', 'target': 'feline', 'type': 'hypernym'}, {'source': 'feline', 'target': 'felid', 'type': 'synonym'}, {'source': 'feline', 'target': 'carnivore', 'type': 'hypernym'}, {'source': 'carnivore', 'target': 'placental', 'type': 'hypernym'}, {'source': 'placental', 'target': 'placental_mammal', 'type': 'synonym'}, {'source': 'placental', 'target': 'eutherian', 'type': 'synonym'}, {'source': 'placental', 'target': 'eutherian_mammal', 'type': 'synonym'}, {'source': 'placental', 'target': 'mammal', 'type': 'hypernym'}, {'source': 'mammal', 'target': 